In [42]:
import pandas as pd
import datetime
from collections import namedtuple
from typing import *

def normalize_ticker(x):
    """ Faz o ticker/ código de negociação ficar o mesmo para BBAS3 e BBAS3F por exemplo.
    """
    if x[-1]=='F':
        return x[:5]
    else:
        return x
        
def normalize_excel(inp):
    """ Passa input de negociação para nomes de variáveis melhores.
    """
    inp['ticker'] = inp['Código de Negociação'].apply(normalize_ticker)
    inp['compravenda'] = inp['Tipo de Movimentação']
    inp['date'] = pd.to_datetime(inp['Data do Negócio'],format='%d/%m/%Y')
    inp['quant'] = inp['Quantidade']
    inp['price'] = inp['Preço']
    inp['value'] = inp['Valor']
    return inp.sort_values(by=['date'])

class SaleReport:
    """ Relatório mensal de vendas indicando se o imposto de renda deve ser olhado.
    A regra é, 15% sobre o lucro das operações caso tenha tido mais de 20 mil reais 
    em vendas de ações no mês.
    Para ETF e Fundos Imobiliários sempre tem o imposto de 15%.
    Caso tenha tido prejuízo acumulado em meses anteriores, pode ser deduzido.

    Aqui o objetivo é só mostrar os meses que precisam da análise se precisa de imposto. Ficando essa outra parte manual.
    
    """
    def __init__(self, atype,period):
        self.profit = 0
        self.tickers = set()
        self.type = atype
        self.period = period
        self.tax_needed=False
        self.total_sales = 0
        
        
    def _repr_pretty_(self,p,cycle):
        return p.text("tickers={} lucro={:.2f} type={} periodo={} olhar_imposto={} total_vendas={:.2f}".format(
            self.tickers, self.profit, self.type,self.period, self.tax_needed, self.total_sales))
        
        

def analyze_sale(accumulated_sales:dict[str,object], r,profit:int):
    """Analisa se a venda é passivel de impostos.
    1. Venda de acoes acima de 20000 no mês com lucro.
    2. Venda de ETF com lucro 
    3. Venda de FII com lucro

    Adiciona o resultado no accumulated_sales para ser retornado no update_wallet como um report.
    Accumulated_sales é um dict, com chave ano/mes-tipo_ativo, e valor um objeto com lista de ativos, lucro/prejuizo, se deve ser observado imposto ou nao.
    """
    asset_type = 'FUNDO' if r.ticker[-2:] =='11' else 'ACAO'
    period = r.date.strftime('%Y-%m')
    last = accumulated_sales.get(period+'-'+asset_type,SaleReport(asset_type,period)) 
    last.tickers.add(r.ticker)
    last.total_sales += r.value
    last.profit += profit
    if asset_type == 'ACAO' and last.total_sales > 20_000 :
        last.tax_needed = last.profit > 0
    elif asset_type == 'FUNDO':
        last.tax_needed = True
    accumulated_sales[period+'-'+asset_type] = last
    return accumulated_sales
         



def update_wallet(walleti,neg):
    wallet = walleti.copy()
    accumulated_sales = dict()
    for i,r in neg.iterrows():
        w = wallet.get(r.ticker)
        if w is None:
            if r.compravenda=='Compra':
                wallet[r.ticker] = {'quant':r.quant,'value':r.value,'PM':r.value/r.quant}
            else:
                raise Exception('Selling something not on wallet '+r.ticker+' quant='+str(r.quant))
        else:
            if r.compravenda=='Compra':
                v = {'quant':r.quant + w['quant'],'value':r.value + w['value']}
                v['PM'] = v['value']/v['quant']
                wallet[r.ticker] = v
            else:
                if r.quant > w['quant']:
                    raise Exception('Selling more than it has '+r.ticker + ' '+str(r.quant)+' - '+str(w['quant']))
                wallet[r.ticker] = {'quant':w['quant']-r.quant,'value': w['value'] - w['PM']*r.quant,'PM':w['PM']}
                profit = r.value - r.quant*w['PM']
                analyze_sale(accumulated_sales,r,profit)
    return wallet, accumulated_sales
    
def load_wallet():
    wallet = pd.read_excel('posicao-anterior.xlsx')
    wallet = wallet.rename(columns={'Código de Negociação':'ticker','Quantidade':'quant','Valor':'value'})
    wallet['PM'] = wallet.value/wallet.quant
    return wallet.set_index('ticker').to_dict(orient='index')

def html_description_line(df):
    return f'{df.quant} ações de {df.ticker} ao Preço Médio de {df.PM}'

def print_html_report(wallet, sales):
    for k,v in wallet.items():
        v['ticker'] = k
    wdf = pd.DataFrame(*[wallet.values()])
    wdf = wdf[wdf.quant!=0]
    from IPython.display import HTML, display,Markdown
    display(Markdown("""# Posição Atual"""))
    display(HTML(wdf.to_html(notebook=True, float_format='{:10.2f}'.format)))
    #display(Markdown("""# Movimentações por mês"""))
    sales_df = pd.DataFrame([{'tickers':list(it.tickers), 'lucro':it.profit,'tipo':it.type,'periodo':it.period,'Olhar imposto':it.tax_needed,'Vendas Totais':it.total_sales } for it in sales.values()])
    display(HTML(sales_df.to_html(notebook=True, float_format='{:10.2f}'.format)))

In [43]:
wallet = load_wallet()
wallet

{'ETER3': {'quant': 100, 'value': 2700, 'PM': 27.0},
 'EZTC3': {'quant': 298, 'value': 5893, 'PM': 19.7751677852349},
 'MXRF11': {'quant': 562, 'value': 5900, 'PM': 10.498220640569395},
 'PETR4': {'quant': 501, 'value': 7500, 'PM': 14.970059880239521}}

In [44]:
trans_df = normalize_excel(pd.read_excel('negociacao-b3.xlsx'))
end_wallet, sales_report = update_wallet(wallet,trans_df)
print_html_report(end_wallet,sales_report)

# Posição Atual

,quant,value,PM,ticker
0,10,270.00,27.00,ETER3
1,298,5893.00,19.78,EZTC3
2,562,5900.00,10.50,MXRF11
3,501,7500.00,14.97,PETR4
4,25,772.25,30.89,BBSE3
5,60,4526.40,75.44,RFOF11


,tickers,lucro,tipo,periodo,Olhar imposto,Vendas Totais
0,"[BBSE3, ETER3]",26811.00,ACAO,2023-11,True,32430.00


In [41]:
trans_df[trans_df.ticker=='ETER3'] # pode editar para analisar transações especificas

,Data do Negócio,Tipo de Movimentação,Mercado,Prazo/Vencimento,Instituição,Código de Negociação,Quantidade,Preço,Valor,ticker,compravenda,date,quant,price,value
4,2023-11-10 00:00:00,Venda,Mercado à Vista,-,XP INVESTIMENTOS CCTVM S/A,ETER3,90,27.0,2430.0,ETER3,Venda,2023-11-10,90,27.0,2430.0


In [46]:
a = 'VRTA11'
a[-2:]

'11'